Création des variables locales à partir des variables brutes des fichiers amplicon et séparation en ensemble d'entraînement et de test.

In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn_qqplot import pplot
import scipy as sp
import sklearn.cluster
import tqdm
from scipy.stats import zscore, kstest, kruskal

Feature creations for the amplicon file. Separation into Train and Test files

In [2]:
path = r"C:\Users\ahmed\MFE project"    
df = pd.read_csv(os.path.join(path, [i for i in os.listdir(path) if i.find("Database_Amplicon")>-1][0]))

In [3]:
df

,Unnamed: 0,contig_id,contig_srt,contig_end,region_id,attributes,gc_count,overlaps,fwd_e2e,rev_e2e,...,cov500x,annee,type,date,num,classe,num2,num3,ordre,longueur
0,0,chr19,1221237,1221332,CHP2_STK11_4,GENE_ID=STK11;Pool=1,58,140,68,66,...,0,2018,NDN,4-12-18,1,OPT,7,83,0,92
1,1,chr10,89711805,89711932,CHP2_PTEN_4,GENE_ID=PTEN;Pool=1,43,534,258,242,...,126,2018,NDN,4-12-18,1,OPT,7,83,0,92
2,2,chr1,162745448,162745576,ON_DDR2_7,GENE_ID=DDR2;Pool=1,52,593,360,219,...,129,2018,NDN,4-12-18,1,OPT,7,83,0,92
3,3,chr17,7577016,7577151,CHP2_TP53_7,GENE_ID=TP53;Pool=1,80,623,283,309,...,136,2018,NDN,4-12-18,1,OPT,7,83,0,92
4,4,chr3,178938788,178938918,CHP2_PIK3CA_8,GENE_ID=PIK3CA;Pool=1,55,746,339,385,...,131,2018,NDN,4-12-18,1,OPT,7,83,0,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72436,72436,chr2,212652720,212652806,CHP2_ERBB4_2,GENE_ID=ERBB4;Pool=1,32,1825,741,1046,...,87,2021,OBL,23-03-21,2,SOPT,48,636,607,92
72437,72437,chr10,89720696,89720747,CHP2_PTEN_7,GENE_ID=PTEN;Pool=1,19,2045,1056,970,...,52,2021,OBL,23-03-21,2,SOPT,48,636,607,92
72438,72438,chr7,116423408,116423492,CHP2_MET_6,GENE_ID=MET;Pool=1,34,2065,1289,714,...,85,2021,OBL,23-03-21,2,SOPT,48,636,607,92
72439,72439,chr4,153250853,153250926,CHP2_FBXW7_2,GENE_ID=FBXW7;Pool=1,28,2161,1049,1079,...,74,2021,OBL,23-03-21,2,SOPT,48,636,607,92


In [16]:
# to correct the different notations of the type (in the amplicon file)
df = df.replace("NAV", "NVA")
df = df.replace("nva", "NVA")

In [17]:
# additional features

df.loc[:, "contig_length"] = df["contig_end"] - df["contig_srt"]+1
df.loc[:, "gc_percent"] = df["gc_count"]/df["contig_length"]
df.loc[:, "cov20_percent"] = df["cov20x"]/df["contig_length"]
df.loc[:, "cov100_percent"] = df["cov100x"]/df["contig_length"]
df.loc[:, "cov500_percent"] = df["cov500x"]/df["contig_length"]
df.loc[:, "fwd_e2e_percent"] = df["fwd_e2e"]/df["fwd_reads"]
df.loc[:, "rev_e2e_percent"] = df["rev_e2e"]/df["rev_reads"]
df.to_csv("Database_AddFeat_Amplicon.csv")

In [18]:
# split dataframe into one train sample and one test sample
def split_df_test_train(df, type):
    from sklearn.model_selection import train_test_split
    ordre_train= []
    ordre_test=[]
    for i in type:
        df_typ = df[df["type"]==i]
        lis = list(dict.fromkeys(df_typ["ordre"].values))
        lis_train, lis_test = train_test_split(lis, test_size=0.3, shuffle = True)
        ordre_train.extend(lis_train)
        ordre_test.extend(lis_test)

    df_train = pd.concat([df[df["ordre"]==i] for i in ordre_train])
    df_train.index = range(len(df_train))

    df_test = pd.concat([df[df["ordre"]==i] for i in ordre_test])
    df_test.index = range(len(df_test))

    return df_train, df_test

In [19]:
# take 70% of the database to train, and 30% to test
type = ["CVC", "OBL", "NDN", "TM", "BAL", "BM"]
df_train, df_test = split_df_test_train(df, type)
df_train.to_csv("addfeat_Train_Amplicon.csv")
df_test.to_csv("addfeat_Test_Amplicon.csv")

Feature creations for the tsv file. Separation into Train and Test files (with respect to the above separation)